<a href="https://colab.research.google.com/github/cannin/gsoc_2023_pytorch_pathway_commons/blob/main/Notebooks/baseline_model_with_acc_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=be69cbd6cac961d2565a00fc6a702431b550d692344d0f407ba5dba77a84af9f
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


# Importing Data and Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
import sklearn

In [ ]:
!wget https://zenodo.org/record/8286179/files/train_set.zip?download=1 -O train_set.zip

In [ ]:
!wget https://zenodo.org/record/8286179/files/test_set.zip?download=1 -O test_set.zip

In [ ]:
!unzip /content/test_set.zip

In [ ]:
!unzip /content/train_set.zip

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type != 'cuda':
    raise SystemError('GPU device not found')

Using device: cuda
Using device: cuda


In [ ]:
X_train = pd.read_csv("train_set/X_train.csv", index_col=0)
y_train = np.loadtxt("train_set/y_train.csv", delimiter=',')
X_test = pd.read_csv("test_set/X_test.csv", index_col=0)
y_test = np.loadtxt("test_set/y_test.csv", delimiter=',')

In [ ]:
X_train.head()

,A1CF,A2M,A4GNT,AAAS,AACS,AADAT,AAK1,AANAT,AARS,AARS2,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-OR-A5J1-01,0.0000,10373.70,0.0000,2225.26,1266.090,121.0370,868.876,1.4409,3520.65,299.712,...,0.0,306.9160,0.0,2180.120,256.004,0.0000,253.602,183.9580,146.9740,596.062
TCGA-OR-A5J2-01,0.0000,9844.91,0.0000,1509.46,981.613,31.0025,1627.030,0.0000,5470.94,408.666,...,0.0,99.4255,0.0,2553.480,104.684,0.4026,270.165,264.1250,438.4640,801.637
TCGA-OR-A5J3-01,0.5925,7201.84,2.3700,1259.67,5182.050,39.1053,1158.350,0.0000,4042.66,291.512,...,0.0,333.5090,0.0,864.465,193.749,1.1850,297.437,90.6532,190.1940,437.269
TCGA-OR-A5J5-01,1.5492,2939.04,0.7746,2993.80,967.467,98.3734,646.785,1.5492,2738.96,533.687,...,0.0,358.6060,0.0,1803.250,472.502,5.4222,146.398,260.2630,840.4340,512.781
TCGA-OR-A5J6-01,0.0000,9586.96,0.5589,1186.45,1479.290,19.5600,1238.990,0.0000,2996.59,414.102,...,0.0,143.6260,0.0,1110.450,282.223,0.0000,384.493,64.2685,63.1508,475.587


In [ ]:
y_train

array([ 44.54745701,  55.13364237,  68.74445212,  11.9998685 ,
        88.86477956,  16.1094125 ,  19.03540783,  44.44882796,
        30.31199658,  18.11486997,  57.53361607,  99.8783575 ,
        69.20472104,  66.24584936,  17.78610645,  76.63477661,
        16.1094125 ,  49.215899  ,  67.59377979,  18.47650985,
        39.25436434,  15.25462735,  36.2626163 , 121.2479863 ,
        12.59164283,  29.8188513 ,  66.50886018,  72.39372719,
        31.23253444,  18.14774633,  27.02436138,  33.82976625,
       108.1303219 ,  32.67909393, 113.9165598 ,  35.57221291,
        16.37242332,  49.0843936 ,  24.62438768,  11.3094651 ,
        46.48716179,  95.17703916, 153.6311931 , 127.4944932 ,
        68.25130684,  44.84334418,  12.85465365,   4.10954401,
       152.1517572 ,  31.79143242,  43.29815564,  28.30653911,
        29.09557156,  28.63530263])

# Modelling

In [ ]:
!pip install flaml

In [ ]:
from flaml import AutoML
import pickle

In [ ]:
# Initialize an AutoML instance
automl = AutoML()
# Specify automl goal and constraint
automl_settings = {
    "time_budget": 300,  # in seconds
    "metric": 'mse',
    "task": 'regression',
}
# Train with labeled input data
automl.fit(X_train=X_train, y_train=y_train,
           **automl_settings)

[flaml.automl.logger: 07-13 13:46:30] {1693} INFO - task = regression
[flaml.automl.logger: 07-13 13:46:30] {1700} INFO - Data split method: uniform
[flaml.automl.logger: 07-13 13:46:30] {1703} INFO - Evaluation method: cv
[flaml.automl.logger: 07-13 13:46:31] {1801} INFO - Minimizing error metric: mse
[flaml.automl.logger: 07-13 13:46:31] {1911} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 07-13 13:46:31] {2221} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 07-13 13:46:34] {2347} INFO - Estimated sufficient time budget=37135s. Estimated necessary time budget=262s.
[flaml.automl.logger: 07-13 13:46:34] {2394} INFO -  at 64.8s,	estimator lgbm's best error=1240.0832,	best estimator lgbm's best error=1240.0832
[flaml.automl.logger: 07-13 13:46:34] {2221} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 07-13 13:46:35] {2394} INFO -  at 66.0s,	estimator lgbm's best error=1240.0832,	b

In [ ]:
'''retrieve best config and best learner'''
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best loss on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 4, 'num_leaves': 4, 'min_child_samples': 7, 'learning_rate': 0.2798674441105101, 'log_max_bin': 6, 'colsample_bytree': 0.9019070144825116, 'reg_alpha': 0.0009765625, 'reg_lambda': 0.09189114905095784}
Best loss on validation data: 1055
Training duration of best run: 0.7882 s


In [ ]:
'''pickle and save the automl object'''
import pickle

with open('automl.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)
'''load pickled automl object'''
with open('automl.pkl', 'rb') as f:
    automl = pickle.load(f)

In [ ]:
y_pred = automl.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('mse', '=', mse)
print('mae', '=', mae)

mse = 694.1132429594142
mae = 20.67632853192966
